# Iranian Elites' Immigration

## 1.Initialization
- Importing libraries
- loading datasets, including:
    - Travel History Data
    - Label (Elite or not?)
    - Gender
    - Birth
    - Konkour
    - Olympiad
- Building datesets

## 1.1 Setting

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import statsmodels as sm
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statistics
from scipy.stats import ttest_1samp
from statsmodels.api import add_constant
import math
from scipy.stats import norm
import os
import matplotlib.pyplot as plt
import timeit
import math

In [2]:
path=r'C:\Users\Marziyeh\Downloads'
os.chdir(path)

## 1.2 Import Data

In [3]:
Data_Part1 = pd.read_excel (r'Travel History, V3.xlsx',sheet_name='Part 1')
Data_Part2 = pd.read_excel (r'Travel History, V3.xlsx',sheet_name='Part 2')
Data=pd.concat([Data_Part1, Data_Part2])

KeyboardInterrupt: 

In [ ]:
Data

In [ ]:
Label=pd.read_excel (r'Label.xlsx',sheet_name='Sheet1')
Label= Label[Label['Elite'] == 1]
Label.drop("Elite",axis=1,inplace=True)

In [ ]:
Konkour= pd.read_excel (r'Konkour.xlsx',sheet_name='Konkour')
Olympiad= pd.read_excel (r'Olympiad 80-96.xlsx',sheet_name='Olympiad 80-96')
Birth_Data= pd.read_excel (r'BirthDate_f.xlsx',sheet_name='Table53')
Gender= pd.read_excel (r'gendef_f.xlsx',sheet_name='gender_f_')

## 1.3 Build Dataset
### 1.3.1 Info Dataset
The info dateset merges merges all detasets except travel history. So, this dataset consists of information about Birth, gender, and performance in konkour and olympiad.

In [ ]:
Info=pd.merge(
    left=Label,
    right=Birth_Data,
    left_on="Label",
    right_on="Label",
    how="left"
)
Info=pd.merge(
    left=Info,
    right=Gender,
    left_on="Label",
    right_on="Label",
    how="left"
)
Info=pd.merge(
    left=Info,
    right=Konkour,
    left_on="Label",
    right_on="Label",
    how="left"
)
Info.drop("Elite",axis=1,inplace=True)
Info=pd.merge(
    left=Info,
    right=Olympiad,
    left_on="Label",
    right_on="Label",
    how="left"
)
Info = Info.rename(columns={'birth_date': 'Birth', 'gender_f': 'Gender','year': 'olympiad_date','field': 'olympiad_field'})
Info

In [ ]:
df=Data.dropna(subset=["تاریخ رخداد"])  
df["سامانه"].value_counts()
df=df.loc[:, ['کد','تاریخ رخداد','سامانه','کشور']]

state=np.zeros(len(df))
for i in range(len(df)):
    if df.iloc[i][2]=="ورود به مرز":
        state[i]=int(0)
    elif df.iloc[i][2]=="خروج از مرز":
        state[i]=int(1)      
state=pd.DataFrame(state)

df.insert(2, "state", state, True)
df.drop('سامانه', inplace=True, axis=1)
df = df.rename(columns={'کد': 'Code', 'تاریخ رخداد': 'Date','کشور': 'Country'})

df['Year']= df['Date'].map(lambda x: str(x)[:-12])    
df['Month']= df['Date'].map(lambda x: str(x)[5:7])  
df['Day']= df['Date'].map(lambda x: str(x)[8:10])  

df=df.drop(["Country"],axis=1)

df['Day']=pd.to_numeric(df['Day'], errors='coerce')
df['Month']=pd.to_numeric(df['Month'], errors='coerce')
df['Year']=pd.to_numeric(df['Year'], errors='coerce')

df['Days']=pd.DataFrame(np.zeros(len(df)))

for i in range(len(df)):
    if df['Month'].iloc[i]<7:
        month_length=31*df['Month'].iloc[i]
    else:
        month_length=31*6+30*(df['Month'].iloc[i]-6)
    df['Days'].iloc[i]=365*(df['Year'].iloc[i]-1385)+df['Day'].iloc[i]+month_length
    
df=df.sort_values(['Code', 'Days'], ascending=[True, True])

df['Year']=pd.to_numeric(df['Year'])

### 1.3.2 State and Date Datasets
The raw Travel History data consists of travel records for our id s. Each row, shows a specific travel record for an id. We need a dataset that integrates all travel dates and status (whether the passenger leaves Iran or ruturns back), the former is Date dataset and the latter is State. 
In Date dataset, each row shows an id and all travel dates. In State dataset, each row shows an id and the status of subsequent travel dates. The two datasets are complementary.

In [ ]:
Date=[]
State=[]
i=0
while i<len(df):
    Date_row=[]
    State_row=[]
    Date_row.append(df['Code'].iloc[i])
    State_row.append(df['Code'].iloc[i])
    j=0
 
    while df['Code'].iloc[i+j]==df['Code'].iloc[i]:
        Date_row.append(df['Days'].iloc[i+j])
        State_row.append(df['state'].iloc[i+j])
        j=j+1
        if i+j+1>len(df):
            break
    Date.append(Date_row)
    State.append(State_row)
    i=i+j

Date=pd.DataFrame(Date).iloc[:,0:100]
State=pd.DataFrame(State).iloc[:,0:100]

In [ ]:
Date

In [ ]:
State

In [ ]:
State.loc[State[0]==1132161].dropna(axis=1)

In [ ]:
df.loc[df['Code']==1132161]

### 1.3.3 Other Datasets
Now, we may use a subset of previous datasets to work on. For instance, the travel history for elites with a background in olympiad.
Please construct the subsequent datesets in related sections. For exmple, please build datasets which are related to olympiad in the Olympiad section.

In [ ]:
Check_Olympiad=pd.merge(
    left=Olympiad,
    right=State,
    left_on="Label",
    right_on=State[0],
    how="left"
)

### 1.3.4 Export and Import Results
For the purpose of speeding up, please save results which reproduction are time consuming in excel files, to avoid the reproduction process. We may reload them later.

In [ ]:
Date.to_excel(r'Date.xlsx', index = False)
State.to_excel(r'State.xlsx', index = False)

In [ ]:
Date= pd.read_excel (r'Date.xlsx',sheet_name='Sheet1')
State= pd.read_excel (r'State.xlsx',sheet_name='Sheet1')

## 2 Define Migrants
- Who is a migrant? We do not have a solid answer for the question. The initial answer is a citizen who is out of a country for more than 400 days in a 2-year period is assumed as a migrant, till he or she returns to the country and stays there such that the number of days exceeds 400 days in the 2-year period, this way we may recognize a negative migration.
- In this section, first we have eveluated different scenarios for defining migrants, and we run multiple sensitivity analysis on Duration Scenarios (for example 2 years in the initial definition), and Acceptable Duration Scenarios (for example 400 days in the initial definition).
- In the last part, we have defined migrants and negative migrants based on our definition.

### 2.1 Sensitivity Analysis on Duration and Thereshold (Algorithm 1: atleast 1-time migration)
In this algorithm, we define a person as a migrant if he or she spends more than x days out of Iran in any y-year period. 

In [ ]:
Scenario_column=[]
Duration_Scenarios=[500,600,700,800]
Acceptable_Duration_Scenarios=[250,300,350,400,450]
for Duration in (Duration_Scenarios):
    start=timeit.default_timer()
    Scenario_row=[]
    for Acceptable_Duration in (Acceptable_Duration_Scenarios):
        Immigrant=0
        i=0
        while i<len(State):
            j=0
            res=0
            Date_Prime=Date.iloc[i][1:]
            State_Prime=State.iloc[i][1:]
            Date_State_Prime=pd.concat([Date.iloc[i][1:], State.iloc[i][1:]], axis=1, join='inner')
            Date_State_Prime=Date_State_Prime.dropna()
            Date_Prime=np.array(Date_State_Prime.iloc[:,0])
            State_Prime=np.array(Date_State_Prime.iloc[:,1])
            while j<len(State_Prime):
                if res==1:
                    break
                Time_Origin=Date_Prime[j]
                for k in range(-Duration,0,20):
                    Starting_point=max(Time_Origin+k,0)
                    Ending_point=Starting_point+Duration
                    index = np.where(np.logical_and(Date_Prime>=Starting_point, Date_Prime<=Ending_point))
                    index=np.array(index)
                    count=0
                    if State_Prime[index[0][0]]==0:
                        count+=Date_Prime[index[0][0]]-Starting_point
                    if State_Prime[index[0][len(index[0])-1]]==1:
                        count+=Ending_point-Date_Prime[index[0][len(index[0])-1]]   
                    for s in range(len(index[0])-1):
                        if State_Prime[index[0][s]]==1:
                            count+=Date_Prime[index[0][s+1]]-Date_Prime[index[0][s]]
                    if count>Acceptable_Duration:
                        res=1
                        Immigrant+=1
                        break 
                j+=1
            i+=1
        Scenario_row.append(Immigrant/109301)   
        stop = timeit.default_timer()
        print(Acceptable_Duration)
        print(Duration)
        print(stop-start)
        print("****")
    Scenario_column.append(Scenario_row)
Scenario_column

In [ ]:
Result=pd.DataFrame(Scenario_column)
Result.columns=[250,300,350,400,450]
Result.index=[500,600,700,800]
Result

In [ ]:
Date.loc[Date[0]==1069620].loc[:,0:15]

### 2.2 Sensitivity Analysis on Duration and Thereshold (Algorithm 2: last 2 year)
In this algorithm, we define a person as a migrant if he or she spends more than x days out of Iran in the last y years.

In [ ]:
# Duration_Scenarios=[500,600,700,800]
# Acceptable_Duration_Scenarios=[250,300,350,400,450]
Duration_Scenarios=[500,600,700,800]
Acceptable_Duration_Scenarios=[250,300,350,400,450]
Starting_point=13505
Scenario_column=[]
for Duration in (Duration_Scenarios):
    start=timeit.default_timer()
    Scenario_row=[]
    for Acceptable_Duration in (Acceptable_Duration_Scenarios):
        Immigrant=0
        i=0
        while i<len(State):
            j=0
            res=0
            Date_Prime=Date.iloc[i][1:]
            State_Prime=State.iloc[i][1:]
            Date_State_Prime=pd.concat([Date.iloc[i][1:], State.iloc[i][1:]], axis=1, join='inner')
            Date_State_Prime=Date_State_Prime.dropna()
            Date_Prime=np.array(Date_State_Prime.iloc[:,0])
            State_Prime=np.array(Date_State_Prime.iloc[:,1])
            Ending_point=Starting_point+Duration
            index = np.where(np.logical_and(Date_Prime>=Starting_point, Date_Prime<=Ending_point))
            index=np.array(index)
            count=0
            if(len(index[0]))==0:
                t=len(Date_Prime)-1
                while Date_Prime[t]>Starting_point and t>0:
                    t=t-1
                if State_Prime[t]==1:
                    res=1
                    Immigrant+=1
            else:          
                if State_Prime[index[0][0]]==0:
                    count+=Date_Prime[index[0][0]]-Starting_point
                if State_Prime[index[0][len(index[0])-1]]==1:
                    count+=Ending_point-Date_Prime[index[0][len(index[0])-1]]   
                for s in range(len(index[0])-1):
                    if State_Prime[index[0][s]]==1:
                        count+=Date_Prime[index[0][s+1]]-Date_Prime[index[0][s]]
                if count>Acceptable_Duration:
                    res=1
                    Immigrant+=1
            i+=1
#             print(i)
#             print(res)
#             print(count)
#             print(Date_Prime)
#             print(State_Prime)
#             print("***")
        Scenario_row.append(Immigrant/109301)   
        stop = timeit.default_timer()
        print(Acceptable_Duration)
        print(Duration)
        print(stop-start)
        print("****")
    Scenario_column.append(Scenario_row)

In [ ]:
Result_Last=pd.DataFrame(Scenario_column)
Result_Last
Result_Last.columns=[250,300,350,400,450]
Result_Last.index=[500,600,700,800]
Result_Last

### 1.3 Sensitivity Analysis on Duration and Percent Thereshold (Algorithm 3: non-rolling 2 year)
In fact, there should be reasonable relation between our Duration_Scenarios and Acceptable_Duration_Scenarios. It is more meaningful to define a time period in the Acceptable_Duration_Scenarios based on shares. For example, we may define a person as a migrant if he or she spends more than 50 percent of a value is Duration_Scenarios out of Iran. This scenario seems more acceptable!
We have eveluated such scenarios in this section.

In [ ]:
Duration_Scenarios=[500,600,700,800]
Acceptable_Duration_Scenarios=[250,300,350,400,450]
Duration_Scenarios=[365,547,730,912,1095]
Starting_point=4015
Scenario_column=[]
length=len(State)
for Duration in (Duration_Scenarios):
    Acceptable_Duration_Scenarios=[0.3*Duration,0.4*Duration,0.5*Duration,0.6*Duration,0.7*Duration]
    print(Acceptable_Duration_Scenarios)
    start=timeit.default_timer()
    Scenario_row=[]
    for Acceptable_Duration in (Acceptable_Duration_Scenarios):
        Immigrant=0
        i=0
        while i<length:
            j=0
            res=0
            Date_Prime=Date.iloc[i][1:]
            State_Prime=State.iloc[i][1:]
            Date_State_Prime=pd.concat([Date.iloc[i][1:], State.iloc[i][1:]], axis=1, join='inner')
            Date_State_Prime=Date_State_Prime.dropna()
            Date_Prime=np.array(Date_State_Prime.iloc[:,0])
            State_Prime=np.array(Date_State_Prime.iloc[:,1])
            Ending_point=Starting_point+Duration
            index = np.where(np.logical_and(Date_Prime>=Starting_point, Date_Prime<=Ending_point))
            index=np.array(index)
            count=0
            if(len(index[0]))==0:
                t=len(Date_Prime)-1
                while Date_Prime[t]>Starting_point and t>0:
                    t=t-1
                if State_Prime[t]==1:
                    res=1
                    Immigrant+=1
            else:          
                if State_Prime[index[0][0]]==0:
                    count+=Date_Prime[index[0][0]]-Starting_point
                if State_Prime[index[0][len(index[0])-1]]==1:
                    count+=Ending_point-Date_Prime[index[0][len(index[0])-1]]   
                for s in range(len(index[0])-1):
                    if State_Prime[index[0][s]]==1:
                        count+=Date_Prime[index[0][s+1]]-Date_Prime[index[0][s]]
                if count>Acceptable_Duration:
                    res=1
                    Immigrant+=1
            i+=1
        Scenario_row.append(Immigrant/length) 
        print(Immigrant/length)
        stop = timeit.default_timer()
        print(Acceptable_Duration)
        print(Duration)
        print(stop-start)
        print("****")
    Scenario_column.append(Scenario_row)

In [ ]:
Final_Result=pd.DataFrame(Scenario_column)
Final_Result

### 1.4 Determine Migrants
Finally, we have defined migrants based on our chosen scenario. The Info Migration dataset define migration and negative migration records for each id.

In [ ]:
Immigrant=[]
Immigrant_Negative=[]
Immigrant_Final_Status=[]
Immigrantion_Date=[]
Duration=730
Acceptable_Duration=400
Migration_Background=[]
Migration_Background_row=[]
Migration_Background_Status=[]
Migration_Background_Status_row=[]
Info_Migration=[]
Info_Migration_row=[]
i=0
while i<len(State):
    Migration_Background_row=[]
    Migration_Background_Status_row=[]
    Info_Migration_row=[]
    Migration_Background_row.append(Date.iloc[i][0])
    Migration_Background_Status_row.append(Date.iloc[i][0])
    Info_Migration_row.append(Date.iloc[i][0])
    j=0
    Date_Prime=Date.iloc[i][1:]
    State_Prime=State.iloc[i][1:]
    Date_State_Prime=pd.concat([Date.iloc[i][1:], State.iloc[i][1:]], axis=1, join='inner')
    Date_State_Prime=Date_State_Prime.dropna()
    Date_Prime=np.array(Date_State_Prime.iloc[:,0])
    State_Prime=np.array(Date_State_Prime.iloc[:,1])
    Status=0
    while j<len(State_Prime):
        if Date_Prime[j]<4745:
            res=0
            Starting_point=Date_Prime[j]
            Ending_point=Starting_point+Duration
            index = np.where(np.logical_and(Date_Prime>=Starting_point, Date_Prime<=Ending_point))
            index=np.array(index)
            count=0
            for s in range(len(index[0])-1):
                if State_Prime[index[0][s]]==1:
                    count+=Date_Prime[index[0][s+1]]-Date_Prime[index[0][s]]
            if State_Prime[index[0][-1]]==1 :
                count+=Ending_point-Date_Prime[index[0][-1]]   
            if count>Acceptable_Duration:
                res=1

            if Status!=res:
                Status=res
                Migration_Background_row.append(Date_Prime[j])
                Migration_Background_Status_row.append(Status)
        j+=1
    if len(Migration_Background_row)>1:
        Info_Migration_row.append(1)
        if len(Migration_Background_row)>2:
            Info_Migration_row.append(1)
        else:
            Info_Migration_row.append(0)
        Info_Migration_row.append(Migration_Background_Status_row[-1])
        Info_Migration_row.append(Migration_Background_row[1])
        Info_Migration_row.append(State_Prime[-1])
        Info_Migration_row.append(math.modf(Migration_Background_row[1]/365)[1]+1+1385-Info.iloc[i][1])
        
    else:
        Info_Migration_row.append('NaN')
        Info_Migration_row.append('NaN')
        Info_Migration_row.append('NaN')
        Info_Migration_row.append('NaN')
        Info_Migration_row.append('NaN')
        Info_Migration_row.append('NaN')

            
    Migration_Background.append(Migration_Background_row)
    Migration_Background_Status.append(Migration_Background_Status_row)
    Info_Migration.append(Info_Migration_row)
    i+=1
    
Info_Migration=pd.DataFrame(Info_Migration)

In [ ]:
Dataset=pd.merge(
    left=Info,
    right=Info_Migration,
    left_on="Label",
    right_on=Info_Migration[0],
    how="left"
)
Dataset.rename(columns = {0:'Label2', 1:'Positive_Migration',2:'Negative_Migration',3:'Final_Status',4:'Migration_Date', 5:'Current_Status', 6:'Migration_Age'}, inplace = True)
Dataset.drop(['Label2'],axis=1,inplace=True)
Dataset

# 3 Migration Summary Statistics

Here, we draw general results. We do not analyse konkour and olympiad data separately. Issues such as migration age (whether the migrant has a contribution in olympiad or konkour) are investigated here.

### 3.1 Migration rate vs year
What is the total positive and negative migration rate in each year?

In [ ]:
Migration_Positive=[]
Migration_Negative=[]
for i in range (len(Migration_Background)):
    if len(Migration_Background[i])>0:
        for j in range(len(Migration_Background[i])):
            if Migration_Background_Status[i][j]==1:
                Migration_Positive.append(math.modf(Migration_Background[i][j]/365)[1]+1385+1)
            elif Migration_Background_Status[i][j]==0:
                Migration_Negative.append(math.modf(Migration_Background[i][j]/365)[1]+1385+1)
  

In [ ]:
plt.hist(Migration_Positive,bins=13,histtype='bar',density=True, 
         stacked=True, 
         edgecolor="#6A9662",
         color="#DDFFDD")

In [ ]:
plt.hist(Migration_Negative,bins=14,density=True, 
         stacked=True, 
         edgecolor="#6A9662",
         color="#DDFFDD")

In [ ]:
Migration_Positive_Counter=np.zeros(15)
Migration_Negative_Counter=np.zeros(15)
for i in range(15):
    Year=1385+i
    Migration_Positive_Counter[i]=Migration_Positive.count(Year)
    Migration_Negative_Counter[i]=Migration_Negative.count(Year)

In [ ]:
# set width of bar
barWidth = 1
fig = plt.subplots(figsize =(12, 8))
 
# # set height of bar
# IT = [12, 30, 1, 8, 22]
# ECE = [28, 6, 16, 5, 10]
# CSE = [29, 3, 24, 25, 17]

# Set position of bar on X axis
br1 = np.arange(len(Migration_Positive_Counter))
#br2 = [x + barWidth for x in br1]

 
# Make the plot
plt.bar(br1, Migration_Positive_Counter, color ='r', width = barWidth,edgecolor ='grey', label ='Positive Migration', alpha=0.8)
#plt.bar(br2, Migration_Negative_Counter, color ='g', width = barWidth,
       # edgecolor ='grey', label ='Negative Migration', alpha=0.8)

# Adding Xticks
#plt.xlabel('Year', fontweight ='bold', fontsize = 15)
plt.xlabel('Year', fontsize = 15)
plt.ylabel('Migrants (Person)', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(Migration_Positive_Counter))],
        ['1386', '1387', '1388', '1389','1390', '1391', '1392', '1393','1394','1395','1396','1397','1398','1399','1400'])
 
plt.legend()
plt.show()

### 3.2 Return back period
For those who have a record of negative migration, how much time have they spent before they return back?

In [ ]:
Migration_Diff=[]
for i in range (len(Migration_Background)):
    if len(Migration_Background[i])>2:
        for j in range(2,len(Migration_Background[i])):
            Migration_Diff.append(Migration_Background[i][j]-Migration_Background[i][j-1])

In [ ]:
plt.hist(Migration_Diff, bins=100)  

### 3.3 Migration Age
How old was the migrant when he or she migrated for the first time?

In [ ]:
Dataset_Age=Dataset.dropna(subset = ['Migration_Date'])
Dataset_Age
Dataset['Migration_Age'].value_counts()[50]
Dataset['Migration_Age'].hist
Counter_Age=np.zeros(30)
for i in range(20,50):
    Counter_Age[i-20]=Dataset['Migration_Age'].value_counts()[i]
plt.hist(Counter_Age)
Counter_Age
Migration_Age_List=Dataset['Migration_Age']
#Migration_Age_List.dropna(inplace=True)
Migration_Age_List=Dataset.dropna(subset = ['Migration_Age'])
Migration_Age=Migration_Age_List['Migration_Age']
Migration_Age
Migration_Age.hist(bins=100)
Dataset['Migration_Age'].value_counts()[]

## 2 Migration vs Konkur Results 
Here, we want to draw conclusions about migrants who have a contribution in konkour.

In [ ]:
Dataset_Konkour=Dataset.dropna(subset = ['konkour_date'])
Dataset_Konkour
Dataset_Konkour.sort_values(by=['konkour_date'])
Dataset_Konkour_Mashmool=Dataset_Konkour.loc[((Dataset_Konkour['konkour_field']=='هنر') & (Dataset_Konkour['konkour_rank'] <= 40)) | ((Dataset_Konkour['konkour_field']=='علوم ریاضی و فنی') & (Dataset_Konkour['konkour_rank'] <= 150)) | ((Dataset_Konkour['konkour_field']=='علوم انسانی') & (Dataset_Konkour['konkour_rank'] <= 100)) | ((Dataset_Konkour['konkour_field']=='علوم تجربی') & (Dataset_Konkour['konkour_rank'] <= 100))]
Dataset_Konkour_Mashmool

In [ ]:
len(Dataset_Konkour.loc[Dataset_Konkour['Positive_Migration'] == 1])/len(Dataset_Konkour)

In [ ]:
Pop_Honar=[801,814,823,801,823,826,829,904,849,952,949,959,929,610,782]
Pop_Riazi=[854,869,852,867,889,870,861,887,916,981,990,988,987,995,990]
Pop_Tajrobi=[846,851,863,855,873,844,871,918,908,998,996,1000,996,992,999]
Pop_Ensani=[820,851,841,838,836,866,855,897,905,998,989,993,992,991,997]

In [ ]:
Data_Konkur_Summary_Honar=Dataset_Konkour.loc[Dataset_Konkour['konkour_field'] =="هنر"].loc[Dataset_Konkour['Positive_Migration'] == 1]
Counter_Honar=np.zeros(15)
Counter_Honar_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Honar[i]=Data_Konkur_Summary_Honar['konkour_date'].value_counts()[Year]
    Counter_Honar_percent[i]=Counter_Honar[i]/Pop_Honar[i]

In [ ]:
Data_Konkur_Summary_Honar_Mashmool=Dataset_Konkour_Mashmool.loc[Dataset_Konkour['konkour_field'] =="هنر"].loc[Dataset_Konkour['Positive_Migration'] == 1]
Counter_Honar_Mashmool=np.zeros(15)
Counter_Honar_Mashmool_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Honar_Mashmool[i]=Data_Konkur_Summary_Honar_Mashmool['konkour_date'].value_counts()[Year]
    Counter_Honar_Mashmool_percent[i]=Counter_Honar_Mashmool[i]/40

In [ ]:
# set width of bar
barWidth = 0.17
fig = plt.subplots(figsize =(12, 8))
 
# # set height of bar
# IT = [12, 30, 1, 8, 22]
# ECE = [28, 6, 16, 5, 10]
# CSE = [29, 3, 24, 25, 17]

# Set position of bar on X axis
br1 = np.arange(len(Counter_Honar))
br2 = [x + barWidth for x in br1]
 
# Make the plot
plt.bar(br1, Counter_Honar_percent, color ='r', width = barWidth,
        edgecolor ='grey', label ='Top 1000 Participants', alpha=0.8)
plt.bar(br2, Counter_Honar_Mashmool_percent, color ='g', width = barWidth,
        edgecolor ='grey', label ='Recognized as Elite by Bonyad', alpha=0.8)


# Adding Xticks
#plt.xlabel('Year', fontweight ='bold', fontsize = 15)
plt.xlabel('Year', fontsize = 15)
plt.ylabel('Migrants-Honar (Percent)', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(Counter_Honar))],
        ['1380', '1381', '1382', '1383', '1384','1385', '1386', '1387', '1388', '1389','1390', '1391', '1392', '1393','1394'])
 
plt.legend()
plt.show()

In [ ]:
Data_Konkur_Summary_Riazi=Dataset_Konkour.loc[Dataset_Konkour['konkour_field'] =="علوم ریاضی و فنی"].loc[Dataset_Konkour['Positive_Migration'] == 1]
Counter_Riazi=np.zeros(15)
#Counter_Riazi_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Riazi[i]=Data_Konkur_Summary_Riazi['konkour_date'].value_counts()[Year]
    Counter_Riazi_percent[i]=Counter_Riazi[i]/Pop_Riazi[i]
    
Data_Konkur_Summary_Riazi_Mashmool=Dataset_Konkour_Mashmool.loc[Dataset_Konkour['konkour_field'] =="علوم ریاضی و فنی"].loc[Dataset_Konkour['Positive_Migration'] == 1]
Counter_Riazi_Mashmool=np.zeros(15)
Counter_Riazi_Mashmool_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Riazi_Mashmool[i]=Data_Konkur_Summary_Riazi_Mashmool['konkour_date'].value_counts()[Year]
    Counter_Riazi_Mashmool_percent[i]=Counter_Riazi_Mashmool[i]/150

In [ ]:
# set width of bar
barWidth = 0.17
fig = plt.subplots(figsize =(12, 8))
 
# # set height of bar
# IT = [12, 30, 1, 8, 22]
# ECE = [28, 6, 16, 5, 10]
# CSE = [29, 3, 24, 25, 17]

# Set position of bar on X axis
br1 = np.arange(len(Counter_Riazi))
br2 = [x + barWidth for x in br1]
 
# Make the plot
plt.bar(br1, Counter_Riazi_percent, color ='r', width = barWidth,
        edgecolor ='grey', label ='Top 1000 Participants', alpha=0.8)
plt.bar(br2, Counter_Riazi_Mashmool_percent, color ='g', width = barWidth,
        edgecolor ='grey', label ='Recognized as Elite by Bonyad', alpha=0.8)


# Adding Xticks
#plt.xlabel('Year', fontweight ='bold', fontsize = 15)
plt.xlabel('Year', fontsize = 15)
plt.ylabel('Migrants-Honar (Percent)', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(Counter_Riazi))],
        ['1380', '1381', '1382', '1383', '1384','1385', '1386', '1387', '1388', '1389','1390', '1391', '1392', '1393','1394'])
 
plt.legend()
plt.show()

In [ ]:
Data_Konkur_Summary_Tajrobi=Dataset_Konkour.loc[Dataset_Konkour['konkour_field'] =="علوم تجربی"].loc[Dataset_Konkour['Positive_Migration'] == 1]
Counter_Tajrobi=np.zeros(15)
Counter_Tajrobi_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Tajrobi[i]=Data_Konkur_Summary_Tajrobi['konkour_date'].value_counts()[Year]
    Counter_Tajrobi_percent[i]=Counter_Tajrobi[i]/Pop_Tajrobi[i]
    
Data_Konkur_Summary_Tajrobi_Mashmool=Dataset_Konkour_Mashmool.loc[Dataset_Konkour['konkour_field'] =="علوم تجربی"].loc[Dataset_Konkour['Positive_Migration'] == 1]
Counter_Tajrobi_Mashmool=np.zeros(15)
Counter_Tajrobi_Mashmool_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Tajrobi_Mashmool[i]=Data_Konkur_Summary_Tajrobi_Mashmool['konkour_date'].value_counts()[Year]
    Counter_Tajrobi_Mashmool_percent[i]=Counter_Tajrobi_Mashmool[i]/100

In [ ]:
Data_Konkur_Summary_Ensani=Dataset_Konkour.loc[Dataset_Konkour['konkour_field'] =="علوم انسانی"].loc[Dataset_Konkour['Positive_Migration'] == 1]
Data_Konkur_Summary_Ensani
Counter_Ensani=np.zeros(15)
Counter_Ensani_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Ensani[i]=Data_Konkur_Summary_Ensani['konkour_date'].value_counts()[Year]
    Counter_Ensani_percent[i]=Counter_Ensani[i]/Pop_Ensani[i]

In [ ]:
Data_Konkur_Summary_Ensani=Dataset_Konkour_Mashmool.loc[Dataset_Konkour['konkour_field'] =="علوم انسانی"].loc[Dataset_Konkour['Positive_Migration'] == 1]
Counter_Ensani=np.zeros(15)
Counter_Ensani_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Ensani[i]=Data_Konkur_Summary_Ensani['konkour_date'].value_counts()[Year]
    Counter_Ensani_percent[i]=Counter_Ensani[i]/Pop_Ensani[i]

In [ ]:
# set width of bar
barWidth = 0.17
fig = plt.subplots(figsize =(12, 8))
 
# # set height of bar
# IT = [12, 30, 1, 8, 22]
# ECE = [28, 6, 16, 5, 10]
# CSE = [29, 3, 24, 25, 17]

# Set position of bar on X axis
br1 = np.arange(len(Counter_Riazi))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
 
# Make the plot
plt.bar(br1, Counter_Riazi, color ='r', width = barWidth,
        edgecolor ='grey', label ='Riazi', alpha=0.8)
plt.bar(br2, Counter_Tajrobi, color ='g', width = barWidth,
        edgecolor ='grey', label ='Tajrobi', alpha=0.8)
plt.bar(br3, Counter_Ensani, color ='b', width = barWidth,
        edgecolor ='grey', label ='Ensani', alpha=0.8)
plt.bar(br4, Counter_Honar, color ='c', width = barWidth,
        edgecolor ='grey', label ='Honar', alpha=0.8)

# Adding Xticks
#plt.xlabel('Year', fontweight ='bold', fontsize = 15)
plt.xlabel('Year', fontsize = 15)
plt.ylabel('Migrants (Person)', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(Counter_Riazi))],
        ['1380', '1381', '1382', '1383', '1384','1385', '1386', '1387', '1388', '1389','1390', '1391', '1392', '1393','1394'])
 
plt.legend()
plt.show()

In [ ]:
A=pd.DataFrame(Counter_Riazi_percent)
B=pd.DataFrame(Counter_Tajrobi_percent)
C=pd.DataFrame(Counter_Ensani_percent)
D=pd.DataFrame(Counter_Honar_percent)

In [ ]:
# set width of bar
barWidth = 0.17
fig = plt.subplots(figsize =(12, 8))
 
# # set height of bar
# IT = [12, 30, 1, 8, 22]
# ECE = [28, 6, 16, 5, 10]
# CSE = [29, 3, 24, 25, 17]

# Set position of bar on X axis
br1 = np.arange(len(Counter_Riazi))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
 
# Make the plot
plt.bar(br1, Counter_Riazi_percent, color ='r', width = barWidth,
        edgecolor ='grey', label ='Riazi', alpha=0.8)
plt.bar(br2, Counter_Tajrobi_percent, color ='g', width = barWidth,
        edgecolor ='grey', label ='Tajrobi', alpha=0.8)
plt.bar(br3, Counter_Ensani_percent, color ='b', width = barWidth,
        edgecolor ='grey', label ='Ensani', alpha=0.8)
plt.bar(br4, Counter_Honar_percent, color ='c', width = barWidth,
        edgecolor ='grey', label ='Honar', alpha=0.8)

# Adding Xticks
#plt.xlabel('Year', fontweight ='bold', fontsize = 15)
plt.xlabel('Year', fontsize = 15)
plt.ylabel('Migrants (Percent)', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(Counter_Riazi))],
        ['1380', '1381', '1382', '1383', '1384','1385', '1386', '1387', '1388', '1389','1390', '1391', '1392', '1393','1394'])
 
plt.legend()
plt.show()

In [ ]:
# set width of bar
barWidth = 0.17
fig = plt.subplots(figsize =(12, 8))
 
# # set height of bar
# IT = [12, 30, 1, 8, 22]
# ECE = [28, 6, 16, 5, 10]
# CSE = [29, 3, 24, 25, 17]

# Set position of bar on X axis
br1 = np.arange(len(Counter_Riazi))
br2 = [x + barWidth for x in br1]
 
# Make the plot
plt.bar(br1, Counter_Riazi_percent, color ='r', width = barWidth,
        edgecolor ='grey', label ='Riazi', alpha=0.8)
plt.bar(br2, Counter_Tajrobi_percent, color ='g', width = barWidth,
        edgecolor ='grey', label ='Tajrobi', alpha=0.8)

# Adding Xticks
#plt.xlabel('Year', fontweight ='bold', fontsize = 15)
plt.xlabel('Year', fontsize = 15)
plt.ylabel('Migrants (Percent)', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(Counter_Riazi))],
        ['1380', '1381', '1382', '1383', '1384','1385', '1386', '1387', '1388', '1389','1390', '1391', '1392', '1393','1394'])
 
plt.legend()
plt.show()

In [ ]:
Reshte=["علوم ریاضی و فنی","علوم تجربی","علوم انسانی","هنر"]

In [ ]:
Age_Matrix=[]
Age_Matrix_Mashmool=[]
for j in range(len(Reshte)):
    Data_Konkur_Migrated=Dataset_Konkour.loc[Dataset_Konkour['konkour_field']==Reshte[j]].loc[Dataset_Konkour['Positive_Migration'] == 1]
    Data_Konkur_Mashmool_Migrated=Dataset_Konkour_Mashmool.loc[Dataset_Konkour_Mashmool['konkour_field']==Reshte[j]].loc[Dataset_Konkour_Mashmool['Positive_Migration'] == 1]
    Counter_Age=np.zeros(20)
    Counter_Age_percent=np.zeros(20)
    Counter_Age_Mashmool=np.zeros(20)
    Counter_Age_Mashmool_percent=np.zeros(20)
    for i in range(18):
        Year=18+i
        Counter_Age[i]=Data_Konkur_Migrated['Migration_Age'].value_counts()[Year]
        Counter_Age_Mashmool[i]=Data_Konkur_Mashmool_Migrated['Migration_Age'].value_counts()[Year]
    Counter_Age_sum=Counter_Age.sum()
    Counter_Age_Mashmool_sum=Counter_Age_Mashmool.sum()
    i=0
    for i in range(18):
        Counter_Age_percent[i]=(Counter_Age[i]/Counter_Age_sum)*100
        Age_Matrix.append(Counter_Age_percent)
    
    Counter_Age_Mashmool_percent[i]=(Counter_Age_Mashmool[i]/Counter_Age_Mashmool_sum)*100
    Age_Matrix_Mashmool.append(Counter_Age_Mashmool_percent)
    

In [ ]:
Age_Matrix_Mashmool=pd.DataFrame(Age_Matrix_Mashmool)
Age_Matrix_Mashmool

In [ ]:
Counter_Age_percent

In [ ]:
Data_Konkur_Migrated

In [ ]:
Reshte[0]

In [ ]:
Counter_Riazi_Age

## 3 Migration vs Olympiad Results

Here, we want to draw conclusions about migrants who have a contribution in konkour.

In [ ]:
Dataset_Olympiad=Dataset.dropna(subset = ['olympiad_date'])
Dataset_Olympiad
Check_Olympiad_Computer=Check_Olympiad.loc[Check_Olympiad['field']=='کامپیوتر']
Check_Olympiad_Computer_90=Check_Olympiad_Computer.loc[Check_Olympiad['year']==1390]
Check_Olympiad_Computer_90.loc[:,2:20]
Check_Olympiad_Computer_90.rename(columns = {'Migration':1}, inplace = True)
Check_Olympiad_Computer_90
Check_Olympiad.rename(columns = {0:'Label2',1:'Migration'}, inplace = True)
Check_Olympiad2=Check_Olympiad.dropna(subset = ['Label2'])
Check_Olympiad2
Check_Olympiad2['Migration'].value_counts()
len(Dataset_Olympiad.loc[Dataset_Olympiad['Positive_Migration']==1])
Dataset_Olympiad['Positive_Migration'].value_counts()
Dataset_Olympiad['Negative_Migration'].value_counts()
Dataset_Olympiad['Final_Status'].value_counts()
Dataset_Olympiad['Current_Status'].value_counts()

In [ ]:
Dataset_Olympiad.sort_values(by=['olympiad_date'])

In [ ]:
Dataset_Olympiad_Riazi=Dataset_Olympiad.loc[Dataset_Olympiad['olympiad_field'] =="ریاضی"].loc[Dataset_Olympiad['Positive_Migration'] == 1]
Counter_Olympiad_Riazi=np.zeros(15)
#Counter_Honar_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Olympiad_Riazi[i]=Dataset_Olympiad_Riazi['olympiad_date'].value_counts()[Year]
    #Counter_Honar_percent[i]=Counter_Honar[i]/Pop_Honar[i]

In [ ]:
Dataset_Olympiad_fizik=Dataset_Olympiad.loc[Dataset_Olympiad['olympiad_field'] =="فیزیک"].loc[Dataset_Olympiad['Positive_Migration'] == 1]
Counter_Olympiad_fizik=np.zeros(15)
#Counter_Honar_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Olympiad_fizik[i]=Dataset_Olympiad_fizik['olympiad_date'].value_counts()[Year]
    #Counter_Honar_percent[i]=Counter_Honar[i]/Pop_Honar[i]

In [ ]:
Dataset_Olympiad_shimi=Dataset_Olympiad.loc[Dataset_Olympiad['olympiad_field'] =="شیمی"].loc[Dataset_Olympiad['Positive_Migration'] == 1]
Counter_Olympiad_shimi=np.zeros(15)
#Counter_Honar_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Olympiad_shimi[i]=Dataset_Olympiad_shimi['olympiad_date'].value_counts()[Year]

In [ ]:
Dataset_Olympiad_zist=Dataset_Olympiad.loc[Dataset_Olympiad['olympiad_field'] =="ادبی"].loc[Dataset_Olympiad['Positive_Migration'] == 1]
Counter_Olympiad_zist=np.zeros(15)
#Counter_Honar_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Olympiad_zist[i]=Dataset_Olympiad_zist['olympiad_date'].value_counts()[Year]

In [ ]:
Dataset_Olympiad_computer=Dataset_Olympiad.loc[Dataset_Olympiad['olympiad_field'] =="کامپیوتر"].loc[Dataset_Olympiad['Positive_Migration'] == 1]
Counter_Olympiad_computer=np.zeros(15)
#Counter_Honar_percent=np.zeros(15)
for i in range(15):
    Year=1380+i
    Counter_Olympiad_computer[i]=Dataset_Olympiad_computer['olympiad_date'].value_counts()[Year]

In [ ]:
# set width of bar
barWidth = 0.17
fig = plt.subplots(figsize =(12, 8))
 
# # set height of bar
# IT = [12, 30, 1, 8, 22]
# ECE = [28, 6, 16, 5, 10]
# CSE = [29, 3, 24, 25, 17]

# Set position of bar on X axis
br1 = np.arange(len(Counter_Olympiad_Riazi))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
br5 = [x + barWidth for x in br4]
 
# Make the plot
plt.bar(br1, Counter_Olympiad_Riazi, color ='r', width = barWidth,
        edgecolor ='grey', label ='Riazi', alpha=0.8)
plt.bar(br2, Counter_Olympiad_fizik, color ='g', width = barWidth,
        edgecolor ='grey', label ='ّFizik', alpha=0.8)
plt.bar(br3, Counter_Olympiad_shimi, color ='b', width = barWidth,
        edgecolor ='grey', label ='Shimi', alpha=0.8)
plt.bar(br4, Counter_Olympiad_zist, color ='c', width = barWidth,
        edgecolor ='grey', label ='Zist', alpha=0.8)
plt.bar(br4, Counter_Olympiad_zist, color ='c', width = barWidth,
        edgecolor ='grey', label ='Zist', alpha=0.8)

# Adding Xticks
#plt.xlabel('Year', fontweight ='bold', fontsize = 15)
plt.xlabel('Year', fontsize = 15)
plt.ylabel('Migrants (Person)', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(Counter_Olympiad_Riazi))],
        ['1380', '1381', '1382', '1383', '1384','1385', '1386', '1387', '1388', '1389','1390', '1391', '1392', '1393','1394'])
 
plt.legend()
plt.show()

In [ ]:
فیزیک، کامپیوتر، ریاضی، نجوم و اخترفیزیک، شیمی، ادبی، زیست شناسی

In [ ]:
import pyodbc

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\Users\Marziyeh\Downloads\Estelam1.mdb;')
cursor = conn.cursor()
cursor.execute('select * from Javab_Taradod')
   
for row in cursor.fetchall():
    print (row)

In [ ]:
msa_drivers = [x for x in pyodbc.drivers() if 'ACCESS' in x.upper()]
print(f'MS-Access Drivers : {msa_drivers}')

In [ ]:
try:
    con_string = r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\Users\Marziyeh\Downloads\Estelam1.mdb;'
    conn = pyodbc.connect(con_string)
    print("Connected To Database")
 
 
 
except pyodbc.Error as e:
    print("Error in Connection", e)

In [ ]:
con_string = r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};' \
             r'DBQ=C:\Users\Marziyeh\Downloads\Estelam1.mdb;'
conn = pyodbc.connect(con_string)

In [ ]:
conn

In [ ]:
import pyodbc

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\Users\Marziyeh\Downloads\Estelam1.mdb;')
cursor = conn.cursor()
cursor.execute('select * from Javab_Taradod')
   
for row in cursor.fetchall():
    print (row)

In [ ]:
# obtain all data in Person table
cursor.execute('SELECT * FROM Javab_Taradod')
values = cursor.fetchall()
columns = [col[0] for col in cursor.description]

# store data in data frame
df = pd.DataFrame(values, columns=columns)
df.head()

In [ ]:
values